# Pre-Photometry before measuring with GALFITM

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from astropy.io import fits

In [46]:
img_name = 'nircam3'
img_dir = f'./io/image/{img_name}/'
img_merged_path = img_dir + 'merged.fits'

match_dir = f'./io/match/{img_name}/'
prep_dir = f'./io/prep/{img_name}/'

## Create Source Catalog

In [47]:
from photutils.segmentation import SegmentationImage
from photutils.segmentation import SourceCatalog
from astropy.wcs import WCS
from astropy.table import Table, hstack

In [48]:
with fits.open(img_merged_path) as hdul:
    segm = SegmentationImage(hdul['SEGMAP'].data)
    w = WCS(hdul['MERGED'].header)
    image_merged_conv = hdul['SCI_BKSUB_CONV'].data

In [49]:
tab_ref_matched_selected = Table.read(match_dir + 'matchlog.csv', format='ascii.csv')
tab_sorted = tab_ref_matched_selected[tab_ref_matched_selected.argsort('label')] # sort by label
tab_sorted = tab_sorted[tab_sorted['label'] != -1]

label_column = np.int64(tab_sorted['label'])
id_column = np.array(tab_sorted['ID'])
ra_column = tab_sorted['RA']
dec_column = tab_sorted['Dec']
comment_column = tab_sorted['comment']
segm_matched = segm.copy()
segm_matched.keep_labels(label_column)

# Do photometry in every band with SourceCatalog

In [50]:
import glob
img_path_list = glob.glob(img_dir + 'hlsp*.fits')
img_path_list.sort()
img_path_list

['./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f115w_dr0.5_i2d.fits',
 './io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f150w_dr0.5_i2d.fits',
 './io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f200w_dr0.5_i2d.fits',
 './io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f277w_dr0.5_i2d.fits',
 './io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f356w_dr0.5_i2d.fits',
 './io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f410m_dr0.5_i2d.fits',
 './io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f444w_dr0.5_i2d.fits']

In [51]:
PIXAR_SR = 2.11539874851881E-14
ZERO_POINT =  -6.10 -2.5 * np.log10(PIXAR_SR)
print(f'ZERO_POINT: {ZERO_POINT}')
def flux_to_mag(flux, zero_point=ZERO_POINT):
    return -2.5 * np.log10(flux) + zero_point

ZERO_POINT: 28.086519392283982


In [52]:
from utils.zeropoint import calc_zpt

In [53]:
tab_phot = Table([id_column, ra_column, dec_column, comment_column], names=['ID', 'RA', 'Dec', 'comment'])
band_list = []
for i,img_path in enumerate(img_path_list):
    sci = fits.getdata(img_path, hdu='SCI_BKSUB')
    err = fits.getdata(img_path, hdu='ERR')
    hdr_pri = fits.getheader(img_path, extname='PRIMARY')
    band = hdr_pri['FILTER'].split(' ')[0]
    key_tab = f'KronPhot_{band}_mag'
    cat_sci = SourceCatalog(sci, segm_matched, error=err)
    flux_list = cat_sci.kron_flux
    fluxerr_list = cat_sci.kron_fluxerr
    re_list = cat_sci.fluxfrac_radius(0.5)
    x, y = cat_sci.xcentroid, cat_sci.ycentroid
    mag_list = []
    for j in range(len(flux_list)):
        x_pix, y_pix, flux = x[j], y[j], flux_list[j]
        zpt = calc_zpt(x_pix, y_pix, 'CR0')[i]
        mag = flux_to_mag(flux, zero_point= zpt)
        mag_list.append(mag)
    tab_phot.add_columns([mag_list, flux_list, fluxerr_list, re_list], names=[key_tab, key_tab.replace('mag', 'flux'), key_tab.replace('mag', 'fluxerr'), key_tab.replace('mag', 'Re')])

In [54]:
tab_phot.write(prep_dir + 'kronphot.csv', format='ascii.csv', overwrite=True)
tab_phot

ID,RA,Dec,comment,KronPhot_F115W_mag,KronPhot_F115W_flux,KronPhot_F115W_fluxerr,KronPhot_F115W_Re,KronPhot_F150W_mag,KronPhot_F150W_flux,KronPhot_F150W_fluxerr,KronPhot_F150W_Re,KronPhot_F200W_mag,KronPhot_F200W_flux,KronPhot_F200W_fluxerr,KronPhot_F200W_Re,KronPhot_F277W_mag,KronPhot_F277W_flux,KronPhot_F277W_fluxerr,KronPhot_F277W_Re,KronPhot_F356W_mag,KronPhot_F356W_flux,KronPhot_F356W_fluxerr,KronPhot_F356W_Re,KronPhot_F410M_mag,KronPhot_F410M_flux,KronPhot_F410M_fluxerr,KronPhot_F410M_Re,KronPhot_F444W_mag,KronPhot_F444W_flux,KronPhot_F444W_fluxerr,KronPhot_F444W_Re
,,,,,,,pix,,,,pix,,,,pix,,,,pix,,,,pix,,,,pix,,,,pix
int64,float64,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
38742,214.77540889,52.81347611,--,27.166752637260778,2.332956827347645,0.4142403017606851,2.170835406727908,26.818787725074664,3.214346321429253,0.4156369851169136,3.2934548062441453,27.076857260629115,2.534339851001922,0.365653223147898,2.557921381905341,26.561645086539112,4.073331186539674,0.4283985264172767,3.225943037693207,26.94860601844018,2.8521039462868303,0.2667456000753611,3.6888548895022617,27.85726926663973,1.2350941089431986,0.18599211128398577,2.8616544037113343,27.709411012869502,1.4152832010484708,0.16116400614029325,3.4876519709535305
38756,214.82577194,52.84964306,--,27.670271096938436,1.4672337832832094,0.2403304073499856,2.5481909563541545,27.10617040311443,2.4668321258938075,0.2917368264261109,4.520135084807203,27.12828576039763,2.417093512719448,0.2715615962365261,4.78750933865883,26.75561660918948,3.4069135476277728,0.33478156586753915,3.540341836964109,27.255463141538332,2.1499209924411833,0.21863028744946372,3.444331450567881,26.71474168019192,3.537619220486825,0.3561354422809581,3.602710344415901,27.04200933573789,2.6170018076309605,0.25175670035331343,3.788791567633531
38843,214.80667889,52.83783111,--,27.040557613708444,2.620503302921546,0.3983845831856647,2.2398062953406077,27.38425489456597,1.9094390455899475,0.3820912056153122,1.8481442417545566,27.29603881194089,2.071057860046439,0.3326274218130844,2.2450712566755815,27.047035188947326,2.604915748360611,0.3235745731739374,2.74129954447666,26.178132280048874,5.799023182265033,0.6856600016484247,2.7648829196626097,26.179285850696232,5.792865120676414,0.6696032499294566,2.9256221739268566,25.38308032945219,12.06078637452431,1.1832250762895447,3.3185478205000605
23409,214.84438,52.86541194,--,26.566669942689593,4.054523107026343,0.5037502708472016,2.872593916817409,26.556903083414362,4.09116055597655,0.4849876103551064,2.9678896251032443,26.427516219666487,4.6089422775274285,0.45210384083946265,3.7849624240241315,25.97643389427983,6.982873896503436,0.579208732420454,4.175980659336097,26.389593371970463,4.772768922189157,0.38532049431662074,4.206169055737225,26.872449665675614,3.0593410258316385,0.2907074802991059,4.052248317914331,26.96642220682819,2.8056847666774996,0.2647193958527293,3.7244447416729165
23457,214.846515,52.86718306,--,26.418907164182613,4.6456329274448045,0.4238489247832448,5.102944465693253,26.57112553831034,4.037918430716901,0.3642041446881532,4.801880125621439,26.49789242915108,4.319672138262838,0.3608184784644414,4.704055252374929,26.224629159669927,5.555921399108588,0.36942195617814316,5.3227215742521246,26.601499233015616,3.926522258519914,0.2823908058408427,4.991171830502492,26.224950702834498,5.554276244388895,0.3501600462079628,8.075530025981534,26.676549081065826,3.664275547770533,0.24630923616321812,5.81267613616776
24080,214.83425611,52.86177306,--,26.362081878308686,4.895251459716727,0.6615205134576455,2.790385866434499,26.05446367626026,6.498637112650197,0.634636628286342,4.288092040079368,26.476195353644002,4.406863669999689,0.5123561749150554,3.081046699350713,25.96256827239429,7.072622005700319,0.6002816620880086,3.999769506628972,26.142299560934312

# Write to FITS file for GALFIT preparation

#### IMGLIST Extension

In [55]:
pregalfit_path = prep_dir + 'pregalfit.fits'

# Create an IMGLIST extension
hdr_img = fits.Header()
hdr_img['EXTNAME'] = 'IMGLIST'

tab_img = Table()
band_list = []
for img_path in img_path_list:
    print(img_path)
    hdr_pri = fits.getheader(img_path, extname='PRIMARY')
    band = hdr_pri['FILTER'].split(' ')[0]
    band_list.append(band)
tab_img.add_columns([band_list,img_path_list], names=['band', 'img_path'])
fits.writeto(pregalfit_path, tab_img.as_array(), header=hdr_img, overwrite=True)

./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f115w_dr0.5_i2d.fits
./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f150w_dr0.5_i2d.fits
./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f200w_dr0.5_i2d.fits
./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f277w_dr0.5_i2d.fits
./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f356w_dr0.5_i2d.fits
./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f410m_dr0.5_i2d.fits
./io/image/nircam3/hlsp_ceers_jwst_nircam_nircam3_f444w_dr0.5_i2d.fits


#### INIPARAM Extension

In [56]:
srccat = SourceCatalog(image_merged_conv, segm_matched, wcs=w, convolved_data=image_merged_conv, progress_bar=True)
srctab = srccat.to_table(columns = ['label', 'xcentroid', 'ycentroid','orientation','elongation','semimajor_sigma','semiminor_sigma'])
srctab['label'] = srctab['label'].astype(str)
kron_rad = srccat.kron_radius*np.sqrt(srccat.semimajor_sigma.value*srccat.semiminor_sigma.value)
srctab.add_column(kron_rad, name='kron_radius')
tab_final = hstack([tab_phot,srctab])
tab_final = tab_final[tab_final.argsort('ID')]

kron_radius:   0%|          | 0/84 [00:00<?, ?it/s]

In [57]:
# Create a INIPARAM extension
hdr_ini = fits.Header()
hdr_ini['EXTNAME'] = 'INIPARAM'
fits.append(pregalfit_path, tab_final.as_array(), header=hdr_ini, overwrite=True)

hdr_segm = fits.Header()
hdr_segm['EXTNAME'] = 'SEGMAP'
fits.append(pregalfit_path, segm.data, header=hdr_segm, overwrite=True)

In [58]:
tab_test = Table.read('./io/prep/nircam3/pregalfit.fits', hdu='INIPARAM')
tab_test

ID,RA,Dec,comment,KronPhot_F115W_mag,KronPhot_F115W_flux,KronPhot_F115W_fluxerr,KronPhot_F115W_Re,KronPhot_F150W_mag,KronPhot_F150W_flux,KronPhot_F150W_fluxerr,KronPhot_F150W_Re,KronPhot_F200W_mag,KronPhot_F200W_flux,KronPhot_F200W_fluxerr,KronPhot_F200W_Re,KronPhot_F277W_mag,KronPhot_F277W_flux,KronPhot_F277W_fluxerr,KronPhot_F277W_Re,KronPhot_F356W_mag,KronPhot_F356W_flux,KronPhot_F356W_fluxerr,KronPhot_F356W_Re,KronPhot_F410M_mag,KronPhot_F410M_flux,KronPhot_F410M_fluxerr,KronPhot_F410M_Re,KronPhot_F444W_mag,KronPhot_F444W_flux,KronPhot_F444W_fluxerr,KronPhot_F444W_Re,label,xcentroid,ycentroid,orientation,elongation,semimajor_sigma,semiminor_sigma,kron_radius
int64,float64,float64,bytes30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes21,float64,float64,float64,float64,float64,float64,float64
21084,214.81618889,52.83324694,0,25.810142540767465,8.138619424297772,0.665124674956606,4.984369892284277,26.154102546702582,5.928799153245956,0.5997836400826098,4.238707064681377,26.240730813936104,5.474134090066844,0.5516650587892723,4.298849299683731,25.433084850825498,11.5179136046657,0.7192372139046346,5.867158525286223,25.750789967796628,8.595908219765255,0.6123745724117368,5.115749132391561,25.402553953850944,11.846394070900448,0.7926817218964874,5.9294470693880434,25.607976381393826,9.804313831280822,0.6658101399157496,5.514543311664017,32235,6247.165929619373,314.61364287796636,76.04687543169383,1.2706128715647707,5.289511000473978,4.162960346812715,7.89570331924656
21722,214.81303694,52.83425694,0,26.69379153587467,3.606543232617649,0.518684990475995,2.51063693594956,26.358141544065816,4.913049494601305,0.5107602753599336,4.28396208143458,26.729899984812665,3.4885725804166343,0.3900563671504603,3.564594734653966,26.755882673737275,3.4060787704032487,0.2736685975866908,4.761242318660703,26.70742411805673,3.5615423239258535,0.2677395355789626,5.651513956997017,25.65969506302568,9.3482373755585,0.7356468588617611,4.328021085769677,26.245686890195604,5.449203173033493,0.4312377957572461,4.351815152205054,32834,6302.224301676149,564.1455373280204,36.77293784017468,1.0275202176374485,3.2444478671048103,3.157551366302736,8.150110216329042
21816,214.86431,52.87108806,not deblended properly size=70,21.838459607910707,315.6631701522491,26.522432459998853,101.50116595417973,21.56745601993098,405.15886789006214,39.70318442180946,109.47473536451919,21.729003058976158,349.1455678217571,30.914760590675,86.9083760350937,22.38848216971229,190.20191667335814,3.167021313920093,63.14725123555107,22.530035103315498,166.95280326497664,3.1200541340135772,59.9118646045356,22.438242524122728,181.68151690628017,3.720511849056833,46.655906029201525,22.544034658746273,164.81392169901153,3.2122197352761157,54.0352397081389,31988,530.6782514061899,583.6674494866996,-82.5997482632197,1.291244260842696,11.76367098837261,9.110337482309788,34.50697413128691
21820,214.80360111,52.82811,0,25.711677816382167,8.911209016434775,0.759757337763663,5.4250427085171875,25.71905592278398,8.850858298790392,0.7334441888714192,5.556352844047447,25.791900582841308,8.276515329519848,0.6946625323481826,4.928575018656315,25.381564030796515,12.077641785654652,0.8435033855599212,5.156959707640565,25.85624394512911,7.80027975560656,0.5692969433669535,4.898416101816928,26.208530403711674,5.638915613742904,0.4537783153058991,5.091337500918606,25.9674121399253,7.041138725666983,0.4482347312600209,6.0308209143897535,32916,7308.855475951031,610.1367053784818,-22.490384621040956,1.541163087349285,6.167168031113998,4.001632326739141,7.4378716225099595
22085,214.84869389,52.86134611,0,26.04722122899714,6.5421314383797675,0.5201700181159405,12.48326268040173,26.219238552064,5.583574768846839,0.5191664294882204,9.11609442415304,25.830995991528546,7.983794692341803,0.5255996923141457,13.174069246543166,25.250769